In [1]:
import pandas as pd
import os

In [2]:
movie_df = pd.read_csv(os.path.join("../../db_tables", "Movie_rows.csv"))

In [3]:
movie_df = movie_df.sort_values(by="voteCount", ascending=False).reset_index(drop=True)
movie_df = movie_df[:5000]
movie_df

,id,title,originalTitle,overview,tagline,status,releaseDate,runtime,budget,revenue,adult,homepage,imdbId,popularity,voteAverage,voteCount,posterPath,backdropPath
0,27205,Inception,Inception,"Cobb, a skilled thief who commits corporate es...",Your mind is the scene of the crime.,Released,2010-07-15 00:00:00,148,160000000,825532764,False,https://www.warnerbros.com/movies/inception,tt1375666,83.952,8.364,34495,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg
1,157336,Interstellar,Interstellar,The adventures of a group of explorers who mak...,Mankind was born on Earth. It was never meant ...,Released,2014-11-05 00:00:00,169,165000000,701729206,False,http://www.interstellarmovie.net/,tt0816692,140.241,8.417,32571,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg
2,155,The Dark Knight,The Dark Knight,Batman raises the stakes in his war on crime. ...,Welcome to a world without rules.,Released,2008-07-16 00:00:00,152,185000000,1004558444,False,https://www.warnerbros.com/movies/dark-knight/,tt0468569,130.643,8.512,30619,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg
3,19995,Avatar,Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the world of Pandora.,Released,2009-12-15 00:00:00,162,237000000,2923706026,False,https://www.avatar.com/movies/avatar,tt0499549,79.932,7.573,29815,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg
4,24428,The Avengers,The Avengers,When an unexpected enemy emerges and threatens...,Some assembly required.,Released,2012-04-25 00:00:00,143,220000000,1518815515,False,https://www.marvel.com/movies/the-avengers,tt0848228,98.082,7.710,29166,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,175574,Free Birds,Free Birds,Two turkeys from opposite sides of the tracks ...,Hang On To Your Nuggets,Released,2013-10-30 00:00:00,91,55000000,110000000,False,NaN,tt1621039,17.418,5.862,714,/gnSU2wUBq2gTkBEkxY8C1d1fXAQ.jpg,/bYCYL5mVX4unStJy34wWuofVq2O.jpg
4996,4787,Cassandra's Dream,Cassandra's Dream,The tale of two brothers with serious financia...,Family is family. Blood is blood.,Released,2007-06-18 00:00:00,108,0,22687639,False,http://www.cassandrasdreammovie.com/,tt0795493,13.531,6.244,714,/hydGWxW9VvRXMwpwFacl7XVtoeR.jpg,/5Xb2e63jvhJqL6IUO2SqtuRIJRG.jpg
4997,659959,Summer of 85,Été 85,What do you dream of when you're 16-years-old ...,NaN,Released,2020-07-14 00:00:00,101,6900000,3600000,False,NaN,tt10457128,12.932,7.408,714,/rkJKDC5gYrPXQx9IqNaUhSb3beC.jpg,/oq1AkrBDj8hRVJqC2NDFT5R1NEa.jpg
4998,1040,The Leopard,Il gattopardo,As Garibaldi's troops begin the unification of...,Luchino Visconti's enduring romantic adventure,Released,1963-03-28 00:00:00,186,0,0,False,NaN,tt0057091,13.924,7.672,714,/riSUxwoK3xjkOgy6YJSvPhi7cO6.jpg,/myikoqu8Z2gtE7TaBgk9r6fB9MF.jpg


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy import sparse
import io
import base64
import json

In [5]:
# overview TF-IDF

movie_df["overview"] = movie_df["overview"].fillna("")
tfidf = TfidfVectorizer(stop_words='english')

overview_tfidf_matrix = tfidf.fit_transform(movie_df["overview"])

sparse.save_npz("../../movie_vectors/overview_tfidf.npz", overview_tfidf_matrix)

In [6]:
movie_vectors = []

for i in range(overview_tfidf_matrix.shape[0]):
    row_csr = overview_tfidf_matrix[i]  # get sparse row
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)   # serialize to bytes
    binary_data = buffer.getvalue()     # get bytes object
    
    movie_vectors.append({
        "movieId": int(movie_df.iloc[i]["id"]),  # your movie ID
        "overview": binary_data                  # bytes ready for bytea
    })

json_data = [
    {"movieId": mv["movieId"], "overview": base64.b64encode(mv["overview"]).decode()}
    for mv in movie_vectors
]

# Save to JSON
with open("../../movie_vectors/overview.json", "w") as f:
    json.dump(json_data, f)

In [7]:
# tagline TF-IDF

movie_df["tagline"] = movie_df["tagline"].fillna("")
tfidf = TfidfVectorizer(stop_words='english')

tagline_tfidf_matrix = tfidf.fit_transform(movie_df["tagline"])

sparse.save_npz("../../movie_vectors/tagline_tfidf.npz", tagline_tfidf_matrix)

In [8]:
movie_vectors = []

for i in range(tagline_tfidf_matrix.shape[0]):
    row_csr = tagline_tfidf_matrix[i]  # get sparse row
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)   # serialize to bytes
    binary_data = buffer.getvalue()     # get bytes object
    
    movie_vectors.append({
        "movieId": int(movie_df.iloc[i]["id"]),  # your movie ID
        "tagline": binary_data                  # bytes ready for bytea
    })

json_data = [
    {"movieId": mv["movieId"], "tagline": base64.b64encode(mv["tagline"]).decode()}
    for mv in movie_vectors
]

# Save to JSON
with open("../../movie_vectors/tagline.json", "w") as f:
    json.dump(json_data, f)

In [9]:
tagline_tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15070 stored elements and shape (5000, 3898)>

In [10]:
# keywords TF-IDF

keyword_df = pd.read_csv(os.path.join("../../db_tables", "Keyword_rows.csv"))
movie_keyword_df = pd.read_csv(os.path.join("../../db_tables", "MovieKeyword_rows.csv"))

top_movie_ids = set(movie_df["id"])
movie_keyword_df_filtered = movie_keyword_df[movie_keyword_df["movieId"].isin(top_movie_ids)]
movie_keyword_df_filtered

mk = movie_keyword_df_filtered.merge(
    keyword_df,
    left_on="keywordId",
    right_on="id",
    how="inner"
)

movie_keywords_text = (
    mk.groupby("movieId")["name"]
      .apply(lambda x: " ".join(x))
      .reset_index()
      .rename(columns={"name": "keywords"})
)

movie_keywords_text = movie_df[["id"]].merge(
    movie_keywords_text,
    left_on="id",
    right_on="movieId",
    how="left"
)

movie_keywords_text["keywords"] = movie_keywords_text["keywords"].fillna("").str.lower()
movie_keywords_text = movie_keywords_text.drop(columns=["movieId"])
movie_keywords_text = movie_keywords_text.rename(columns={"id": "movieId"}) 

movie_keywords_text

,movieId,keywords
0,27205,california airplane france kidnapping architec...
1,157336,space space station time warp family relations...
2,155,crime fighter district attorney chaos super po...
3,19995,space soldier futuristic nature alien space co...
4,24428,aftercreditsstinger alien invasion superhero t...
...,...,...
4995,175574,freedom holiday duringcreditsstinger thanksgiving
4996,4787,parent child relationship money love triangle ...
4997,659959,france boys' love (bl) summer job based on nov...
4998,1040,decadence pastor country estate monarchy paler...


In [11]:
tfidf = TfidfVectorizer(
    stop_words="english",
    token_pattern=r"(?u)\b\w+\b"  # good for short keywords
)

keywords_tfidf_matrix = tfidf.fit_transform(movie_keywords_text["keywords"])

sparse.save_npz("../../movie_vectors/keywords_tfidf.npz", keywords_tfidf_matrix)

movie_vectors = []
for i in range(keywords_tfidf_matrix.shape[0]):
    row_csr = keywords_tfidf_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_keywords_text.iloc[i]["movieId"]),
        "keywords": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "keywords": base64.b64encode(mv["keywords"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/keywords.json", "w") as f:
    json.dump(json_data, f)

In [12]:
keywords_tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 83227 stored elements and shape (5000, 8403)>

In [13]:
# cast TF-IDF

movie_person_df = pd.read_csv(os.path.join("../../db_tables", "MovieCast_rows.csv"))

top_movie_ids = set(movie_df["id"])
movie_person_df_filtered = movie_person_df[movie_person_df["movieId"].isin(top_movie_ids)]

movie_person = (
    movie_person_df_filtered.groupby("movieId")["personId"]
      .apply(lambda x: " ".join(x.astype(str)))
      .reset_index()
      .rename(columns={"personId": "persons"})
)

movie_person = movie_df[["id"]].merge(
    movie_person,
    left_on="id",
    right_on="movieId",
    how="left"
)

movie_person["persons"] = movie_person["persons"].fillna("")
movie_person = movie_person.drop(columns=["movieId"])
movie_person = movie_person.rename(columns={"id": "movieId"}) 

movie_person

,movieId,persons
0,27205,2524 3899 6193 24045 27578
1,157336,1813 1893 3895 10297 83002
2,155,1579 1810 3894 3895 6383
3,19995,8691 10205 17647 32747 65731
4,24428,103 1245 3223 16828 74568
...,...,...
4995,175574,887 1752 17782 56322 57755
4996,4787,207 3061 39459 39658 72466
4997,659959,5077 26100 1859007 1871464 2496534
4998,1040,4959 4962 13784 15135 15136


In [14]:
tfidf = TfidfVectorizer()

cast_tfidf_matrix = tfidf.fit_transform(movie_person["persons"])

sparse.save_npz("../../movie_vectors/cast_tfidf.npz", cast_tfidf_matrix)

movie_vectors = []
for i in range(cast_tfidf_matrix.shape[0]):
    row_csr = cast_tfidf_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_person.iloc[i]["movieId"]),
        "cast": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "cast": base64.b64encode(mv["cast"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/cast.json", "w") as f:
    json.dump(json_data, f)

In [15]:
# director TF-IDF

movie_director_df = pd.read_csv(os.path.join("../../db_tables", "MovieDirector_rows.csv"))

top_movie_ids = set(movie_df["id"])
movie_director_df_filtered = movie_director_df[movie_director_df["movieId"].isin(top_movie_ids)]

movie_director = (
    movie_director_df_filtered.groupby("movieId")["personId"]
      .apply(lambda x: " ".join(x.astype(str)))
      .reset_index()
      .rename(columns={"personId": "persons"})
)

movie_director = movie_df[["id"]].merge(
    movie_director,
    left_on="id",
    right_on="movieId",
    how="left"
)

movie_director["persons"] = movie_director["persons"].fillna("")
movie_director = movie_director.drop(columns=["movieId"])
movie_director = movie_director.rename(columns={"id": "movieId"}) 

movie_director

,movieId,persons
0,27205,525
1,157336,525
2,155,525
3,19995,2710
4,24428,12891
...,...,...
4995,175574,8029
4996,4787,1243
4997,659959,4387
4998,1040,15127


In [16]:
tfidf = TfidfVectorizer()

director_tfidf_matrix = tfidf.fit_transform(movie_director["persons"])

sparse.save_npz("../../movie_vectors/director_tfidf.npz", director_tfidf_matrix)

movie_vectors = []
for i in range(director_tfidf_matrix.shape[0]):
    row_csr = director_tfidf_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_director.iloc[i]["movieId"]),
        "director": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "director": base64.b64encode(mv["director"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/director.json", "w") as f:
    json.dump(json_data, f)

In [17]:
# genre multi-hot encoding

movie_genre_df = pd.read_csv(os.path.join("../../db_tables", "MovieGenre_rows.csv"))

top_movie_ids = set(movie_df["id"])
movie_genre_df_filtered = movie_genre_df[movie_genre_df["movieId"].isin(top_movie_ids)]

movie_genre = (
    movie_genre_df_filtered.groupby("movieId")["genreId"]
    .apply(lambda x: " ".join(x.astype(str))) 
    .reset_index()
    .rename(columns={"genreId": "genres"})
)

movie_genre = movie_df[["id"]].merge(
    movie_genre,
    left_on="id",
    right_on="movieId",
    how="left"
)

movie_genre["genres"] = movie_genre["genres"].fillna("")
movie_genre = movie_genre.drop(columns=["id"])
movie_genre["genres"] = movie_genre["genres"].apply(lambda x: x.split() if x else [])
movie_genre

,movieId,genres
0,27205,"[15, 17, 21]"
1,157336,"[11, 15, 21]"
2,155,"[11, 13, 17, 28]"
3,19995,"[14, 15, 17, 21]"
4,24428,"[15, 17, 21]"
...,...,...
4995,175574,"[16, 18, 23]"
4996,4787,"[11, 13, 28]"
4997,659959,"[9, 11]"
4998,1040,[11]


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(movie_genre["genres"])

genre_sparse_matrix = sparse.csr_matrix(genre_matrix)
sparse.save_npz("../../movie_vectors/genres_multihot.npz", genre_sparse_matrix)

movie_vectors = []
for i in range(genre_sparse_matrix.shape[0]):
    row_csr = genre_sparse_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_genre.iloc[i]["movieId"]),
        "genres": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "genres": base64.b64encode(mv["genres"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/genres.json", "w") as f:
    json.dump(json_data, f)

In [19]:
# original language multi-hot encoding

movie_language_df = pd.read_csv(os.path.join("../../db_tables", "MovieOriginalLanguage_rows.csv"))

top_movie_ids = set(movie_df["id"])
movie_language_df_filtered = movie_language_df[movie_language_df["movieId"].isin(top_movie_ids)]

movie_language = (
    movie_language_df_filtered.groupby("movieId")["languageCode"]
    .apply(lambda x: " ".join(x.astype(str))) 
    .reset_index()
    .rename(columns={"languageCode": "originalLanguages"})
)

movie_language = movie_df[["id"]].merge(
    movie_language,
    left_on="id",
    right_on="movieId",
    how="left"
)

movie_language["originalLanguages"] = movie_language["originalLanguages"].fillna("")
movie_language = movie_language.drop(columns=["id"])
movie_language["originalLanguages"] = movie_language["originalLanguages"].apply(lambda x: x.split() if x else [])
movie_language

,movieId,originalLanguages
0,27205,[en]
1,157336,[en]
2,155,[en]
3,19995,[en]
4,24428,[en]
...,...,...
4995,175574,[en]
4996,4787,[en]
4997,659959,[fr]
4998,1040,[it]


In [20]:
mlb = MultiLabelBinarizer()
language_matrix = mlb.fit_transform(movie_language["originalLanguages"])

language_sparse_matrix = sparse.csr_matrix(language_matrix)
sparse.save_npz("../../movie_vectors/language_multihot.npz", language_sparse_matrix)

movie_vectors = []
for i in range(language_sparse_matrix.shape[0]):
    row_csr = language_sparse_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_language.iloc[i]["movieId"]),
        "languages": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "originalLanguage": base64.b64encode(mv["languages"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/languages.json", "w") as f:
    json.dump(json_data, f)

In [21]:
# runtine scaling
from sklearn.preprocessing import MinMaxScaler

movie_df["runtime"] = movie_df["runtime"].fillna(0)
scaler = MinMaxScaler()
runtime_scaled = scaler.fit_transform(movie_df[["runtime"]])

runtime_scaled

array([[0.5942623 ],
       [0.68032787],
       [0.61065574],
       ...,
       [0.40163934],
       [0.75      ],
       [0.36885246]], shape=(5000, 1))

In [22]:
runtime_sparse_matrix = sparse.csr_matrix(runtime_scaled) 
sparse.save_npz("../../movie_vectors/runtime_scaled.npz", runtime_sparse_matrix)

movie_vectors = []
for i in range(runtime_sparse_matrix.shape[0]):
    row_csr = runtime_sparse_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_df.iloc[i]["id"]),
        "runtime": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "runtime": base64.b64encode(mv["runtime"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/runtime.json", "w") as f:
    json.dump(json_data, f)

In [23]:
# voteCount scaling

movie_df["voteCount"] = movie_df["voteCount"].fillna(0)
scaler = MinMaxScaler()
voteCount_scaled = scaler.fit_transform(movie_df[["voteCount"]])

voteCount_scaled

array([[1.        ],
       [0.94304491],
       [0.88526095],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]], shape=(5000, 1))

In [24]:
voteCount_sparse_matrix = sparse.csr_matrix(voteCount_scaled) 
sparse.save_npz("../../movie_vectors/voteCount_scaled.npz", voteCount_sparse_matrix)

movie_vectors = []
for i in range(voteCount_sparse_matrix.shape[0]):
    row_csr = voteCount_sparse_matrix[i]
    buffer = io.BytesIO()
    sparse.save_npz(buffer, row_csr)
    binary_data = buffer.getvalue()
    
    movie_vectors.append({
        "movieId": int(movie_df.iloc[i]["id"]),
        "voteCount": binary_data
    })

json_data = [
    {"movieId": mv["movieId"], "voteCount": base64.b64encode(mv["voteCount"]).decode()}
    for mv in movie_vectors
]

with open("../../movie_vectors/voteCount.json", "w") as f:
    json.dump(json_data, f)

In [30]:
# combined movie vectors with weighted features

import numpy as np

weights = {
    "overview": 3.0,
    "keywords": 2.5,
    "genres": 2.5,
    "director": 2.0,
    "cast": 1.5,
    "tagline": 1.5,
    "originalLanguage": 1.0,
    "runtime": 0.5,
    "voteCount": 0.5
}

feature_files = {
    "overview": "../../movie_vectors/overview_tfidf.npz",
    "keywords": "../../movie_vectors/keywords_tfidf.npz",
    "genres": "../../movie_vectors/genres_multihot.npz",
    "director": "../../movie_vectors/director_tfidf.npz",
    "cast": "../../movie_vectors/cast_tfidf.npz",
    "tagline": "../../movie_vectors/tagline_tfidf.npz",
    "originalLanguage": "../../movie_vectors/language_multihot.npz",
    "runtime": "../../movie_vectors/runtime_scaled.npz",
    "voteCount": "../../movie_vectors/voteCount_scaled.npz"
}

matrices = {feature: sparse.load_npz(path).astype(np.float32) for feature, path in feature_files.items()}

weighted_matrices = {feature: matrices[feature] * weights[feature] for feature in weights}

movie_ids = movie_df["id"].tolist()

combined_json = []

for i, movie_id in enumerate(movie_ids):
    movie_dict = {"movieId": int(movie_id)}
    row_vectors_combined = []

    for feature in ["overview", "keywords", "genres", "director", "cast", "tagline", "originalLanguage", "runtime", "voteCount"]:
        row_csr = matrices[feature][i] 
        buffer = io.BytesIO()
        sparse.save_npz(buffer, row_csr)
        binary_data = buffer.getvalue()
        movie_dict[feature] = base64.b64encode(binary_data).decode()

        row_vectors_combined.append(weighted_matrices[feature][i])

    # Create combined weighted vector
    combined_vector = sparse.hstack(row_vectors_combined).tocsr()
    buffer = io.BytesIO()
    sparse.save_npz(buffer, combined_vector)
    movie_dict["combinedVector"] = base64.b64encode(buffer.getvalue()).decode()

    combined_json.append(movie_dict)

# Save final JSON
with open("../../movie_vectors/combined_movie_vectors.json", "w") as f:
    json.dump(combined_json, f)

print("Combined movie vectors JSON with weighted combinedVector created successfully!")

Combined movie vectors JSON with weighted combinedVector created successfully!


In [35]:
combined_json[0]

{'movieId': 27205,
 'overview': 'UEsDBC0AAAAIAAAAIQCs7gb4//////////8LABQAaW5kaWNlcy5ucHkBABAA3AAAAAAAAACUAAAAAAAAAJvsF+obEMnIUMZQrZ6SWpxcpG6loG6TaaKuo6Cell9UUpSYF59flJIKEndLzClOBYoXZyQWpAL5GkbGOpo6CrUK5AOu6dwMDMq8DAwHgbiHj4FBn5+BwUOcgUFYkYFhIhAvAOJTQGykxMDgpg4U12dgMARiB0OgHiA+Z8bAEG/LwLDdgYEh14mBoQCIS4E4wpmBAQBQSwMELQAAAAgAAAAhADhex4b//////////woAFABpbmRwdHIubnB5AQAQAIgAAAAAAAAASQAAAAAAAACb7BfqGxDJyFDGUK2eklqcXKRupaBuk2mirqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGOpo6CrUKFAAuBiAQB2IAUEsDBC0AAAAIAAAAIQCjw3SX//////////8KABQAZm9ybWF0Lm5weQEAEACDAAAAAAAAAEUAAAAAAAAAm+wX6hsQychQxlCtnpJanFykbqWgXhNsrK6joJ6WX1RSlJgXn1+UkgoSd0vMKU4FihdnJBakAvkamjoKtQoUAa7k4iIAUEsDBC0AAAAIAAAAIQDyjt0q//////////8JABQAc2hhcGUubnB5AQAQAJAAAAAAAAAASwAAAAAAAACb7BfqGxDJyFDGUK2eklqcXKRupaBuk2mhrqOgnpZfVFKUmBefX5SSChJ3S8wpTgWKF2ckFqQC+RpGOpo6CrUKFAAuRgYIMPKG0ABQSwMELQAAAAgAAAAhAOroKgT//////////wgAFABkYXRhLm5weQEAEADcAAAAAAAAAJgAAAAAAAAAm+wX6hsQychQxlCtnpJanFykbqWgbpNmoq6joJ6WX1RSlJgXn1+UkgoSd0vMKU4FihdnJBakAvkaRsY6mjo